In [1]:
%load_ext autoreload
%autoreload 2
    

import pickle
import numpy as np 
from matplotlib import pyplot as plt
import itertools

from src.utils.generate_query import generate_queries
from src.blast.hashtable_generation_inference import consensus_seq_match

In /home/NOBACKUP/mzhou3/ming_env/ming_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/NOBACKUP/mzhou3/ming_env/ming_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/NOBACKUP/mzhou3/ming_env/ming_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/NOBACKUP/mzhou3/ming_env/ming_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases

In [2]:
entry = {'pos': 360468,
 'query': 'TAAACCCCGGGCCATATTATGCTGGGAGGCTCACCTCTGTAATGTCGGGGTCTTGGAGGCTGAGGAGGTATTGCTTTGGGGAGTTCAAGCCCAGCCTGGG'}


# Step 1 seed matching

In [22]:
table_data = pickle.load(open('src/data/hashtable.p','rb'))
label_pos = entry['pos']
query = entry['query']    

In [27]:
outputs_step1 = consensus_seq_match(table_data, query)

print(outputs_step1[0])

{'query_idx': 3, 'seed_matching_result': {'ref_idx': 12257, 'score': 0}}


# Step 2 Ungapped extension

In [28]:
from src.blast.ungapped_extension import ungapped_extension

In [29]:
outputs_step2 = ungapped_extension(query, outputs_step1, 'src/data/reference_matrix.p', 11, 1, 'sum_proba_score_correct0')

In [30]:
outputs_step2 = sorted(outputs_step2, key=lambda x:x['ungapped_extension_result']['score'] * -1)

In [37]:
for idx, entry in enumerate(outputs_step2):
    
    seed_correct = (entry['query_idx'] + label_pos) == entry['seed_matching_result']['ref_idx']
    score = entry['ungapped_extension_result']['score']
    span = entry['ungapped_extension_result']['query_right_idx']  - entry['ungapped_extension_result']['query_left_idx'] + 1
    print('{3}: seed correct {0},   scorre {1} ,   span {2}  '.format(seed_correct, float(round(score,2)), span, idx))

0: seed correct True,   scorre 7.33 ,   span 24  
1: seed correct True,   scorre 6.42 ,   span 27  
2: seed correct True,   scorre 5.68 ,   span 27  
3: seed correct True,   scorre 5.68 ,   span 27  
4: seed correct True,   scorre 4.86 ,   span 27  
5: seed correct True,   scorre 4.58 ,   span 27  
6: seed correct True,   scorre 3.62 ,   span 27  
7: seed correct False,   scorre 2.92 ,   span 14  
8: seed correct False,   scorre 2.92 ,   span 14  
9: seed correct False,   scorre 2.88 ,   span 14  
10: seed correct False,   scorre 2.88 ,   span 14  
11: seed correct False,   scorre 2.88 ,   span 14  
12: seed correct False,   scorre 2.68 ,   span 14  
13: seed correct False,   scorre 2.64 ,   span 14  
14: seed correct False,   scorre 2.08 ,   span 14  
15: seed correct False,   scorre 1.97 ,   span 17  
16: seed correct False,   scorre 1.59 ,   span 20  
17: seed correct False,   scorre 0.98 ,   span 12  
18: seed correct False,   scorre 0.98 ,   span 12  
19: seed correct False,   sco

# Step 3 gapped extension

In [38]:
from src.blast.gapped_extension_j import gapped_extension as gapped_extension_1
from src.blast.gapped_extension_j_mingmodified import gapped_extension as gapped_extension_2


In [39]:
gap_bias= -1
gap_penalty = -100
mismatch_score = 1
score_method = "sum_proba_score"
substitution_dict = {}


In [92]:
outputs_step3 = gapped_extension_2(query, 'src/data/reference_matrix.p', outputs_step2, 'sum_proba_score', {}, -1, -1, 1)

In [96]:
outputs_step3[0]

{'query_idx': 78,
 'seed_matching_result': {'ref_idx': 360546, 'score': 0},
 'ungapped_extension_result': {'query_left_idx': 76,
  'query_right_idx': 99,
  'ref_left_idx': 360544,
  'ref_right_idx': 360567,
  'score': 7.326666666666666},
 'gapped_extension_result': {'ref_left_idx': 360296,
  'ref_right_idx': 360567,
  'score': -135.6266666666667,
  'left_alignment_result': {'ref_left_idx': 360296,
   'ref_right_idx': 360543,
   'score': -135.6266666666667,
   'query_string': 'T-------------------------AA-ACC-CC----GGG--CC-----------ATATT----------ATG------CTG-----------------------------------GGAG-----------------------------------------------------------------------------GCTCACCTCTGTAATGTCGGGGTCTTGGAGGCTGAGGAGGTATTGCTT',
   'ref_string': 'TAGAAGAAAAAAAAAATCCACAGCCAAATACCTCCTGCAGGGTACCAGTCAAGCTAAATATTCTTAGTATACATGAAGTATCTGATTACCAGAAGGTGGTTGCAAGTTTATCTTTTTTAGGAGTTCCAAGTAAAGCAATTAGGAAAACCAAGTATACCAGCAATTGAAAATGTAAACCCGGAGCCATAATAGGCTGGGTGGCTCACGTCTGTAATCCCTGCGTCTTGGAGGCTGAGGCGGGATTGGGG'}

test gapped extension method: old vs new

In [90]:
test_case = [outputs_step2[4]]

In [86]:
ungapped_dict = {(45, 71): [[[360513, 360539], 4.86]]}

In [89]:
gapped_extension_2(query, 'src/data/reference_matrix.p', test_case, 'sum_proba_score', {}, -1, -1, 1)

[{'query_idx': 52,
  'seed_matching_result': {'ref_idx': 360520, 'score': 0},
  'ungapped_extension_result': {'query_left_idx': 45,
   'query_right_idx': 71,
   'ref_left_idx': 360513,
   'ref_right_idx': 360539,
   'score': 4.86},
  'gapped_extension_result': {'ref_left_idx': 360296,
   'ref_right_idx': 360567,
   'score': -131.96000000000006,
   'left_alignment_result': {'ref_left_idx': 360296,
    'ref_right_idx': 360512,
    'score': -146.74000000000004,
    'query_string': 'T-------------------------AA-ACC-CC----GGG--CC-----------ATATT----------ATG------CTG-----------------------------------GGAG-----------------------------------------------------------------------------GCTCACCTCTGTAATGT',
    'ref_string': 'TAGAAGAAAAAAAAAATCCACAGCCAAATACCTCCTGCAGGGTACCAGTCAAGCTAAATATTCTTAGTATACATGAAGTATCTGATTACCAGAAGGTGGTTGCAAGTTTATCTTTTTTAGGAGTTCCAAGTAAAGCAATTAGGAAAACCAAGTATACCAGCAATTGAAAATGTAAACCCGGAGCCATAATAGGCTGGGTGGCTCACGTCTGTAATCC'},
   'right_alignment_result': {'ref_left_idx': 360540,
  

In [87]:
gapped_extension_1(query, 'src/data/reference_matrix.p', ungapped_dict
                    , 'sum_proba_score', {}, -1, -1, 1)

{(360296,
  360567): ['T-------------------------AA-ACC-CC----GGG--CC-----------ATATT----------ATG------CTG-----------------------------------GGAG-----------------------------------------------------------------------------GCTCACCTCTGTAATGTCGGGGTCTTGGAGGCTGAGGAGGTATTGCTTTGGGGAGTTCAAGCCCAGCCTGGG', 'TAGAAGAAAAAAAAAATCCACAGCCAAATACCTCCTGCAGGGTACCAGTCAAGCTAAATATTCTTAGTATACATGAAGTATCTGATTACCAGAAGGTGGTTGCAAGTTTATCTTTTTTAGGAGTTCCAAGTAAAGCAATTAGGAAAACCAAGTATACCAGCAATTGAAAATGTAAACCCGGAGCCATAATAGGCTGGGTGGCTCACGTCTGTAATCCCTGCGTCTTGGAGGCTGAGGCGGGATTGGGGTTGGGAGTTCAAGACCAGCCTGGG', -127.10000000000005]}